In [82]:
import pandas
import requests
import json
import re

In [70]:
books = pandas.read_csv('../../Sources/AI/bookai/Books.csv', 
                        names = ['ISBN', 'title', 'author', 'year_publication', 'publisher', 'img_s', 'img_m', 'img_l'], 
                        skiprows = 1, 
                        low_memory=False)

In [114]:
urlMultipleInsert = "http://bookmark-bookshelf.herokuapp.com/api/book/multiple"

def parseTitle (title):
    b = {}
    
    match1 = re.search("^(.*) \(((.*) (Series)?)\)", title)
    if match1:
        b["book_name"] = match1[1]
        b["series_name"] = match1[2]
    
    match2 = re.search("^(.*) \((.*), Book (\d)+\)", title)
    if match2:
        b["book_name"] = match2[1]
        b["series_name"] = match2[2]
        b["series_position"] = match2[3]
        
    if not match1 and not match2:
        b["book_name"] = title

    return b

def bookArray (index):
    b_data = parseTitle(books.title[index])
    b_data["isbn13"] = books.ISBN[index]
    b_data["authors"] = [books.author[index]]
    b_data["cover"] = books.img_l[index]
    
    return b_data

def getXBooks (x, start_i):
    lst = []
    for i in range(x):
        try:
            lst.append(bookArray(start_i + i))
        except:
            print("Error at index", start_i + i)
    return lst



nb_books = len(books.index)
print("Number of books: ", nb_books)

Number of books:  271360


In [110]:
def sendRequest (url, data):
    response = requests.post(url, json=data)
    if response.status_code != 200:
        print(data)
        print(response.text)

In [117]:
step = 20
start = 0
for i in range(5):
    books_data = getXBooks(step, start)
    sendRequest(urlMultipleInsert, books_data)
    start = start + step